# FindYourLandlord Data Cleaning
### This project aims to:
* provide a visual representation for who or what entity owns the most property, what other properties the owner owns, and where
* provide tenants in North Jersey with pertinent information about who or what owns their building or apartment
### Beginning with Jersey City, I'll eventually expand to other cities throughout North Jersey: Paterson, Newark, Hackensack…
### This notebook is the data portion of the project, where I will test out methods for cleaning and extracting the relevant data. The data will eventually be picked up by React and mapboxGL.
### The csv's were retrieved from Monmouth County's Tax Assessor website:
### https://tax1.co.monmouth.nj.us/cgi-bin/prc6.cgi?menu=index&ms_user=monm&passwd=data&district=1301&mode=11

## To Do:
* the data is unbelievably irregular: needs cleaning and normalization
* create column of integers representing the number of unique properties associated with the owner: this informs the size of the circle's radius on the map
* create column of an array of strings of the properties associated with the owner: this informs the size of the bounding box of selected properties on the map
* units: if the cells in the column of string arrays has a len of 1, but the column of integers is < 1, it represents the number of units at the property

In [1]:
import pandas as pd
import os
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# for returning lat long coordinates from addresses
geolocator = Nominatim(user_agent="myGeolocator", timeout=2)

In [2]:
hackensack = pd.read_csv('0223monm205221.csv', dtype={14: str, 20: str, 24: str})

jersey_city = pd.read_csv('0906monm220028.csv', dtype={20: str,
                                                       22: str,
                                                       23: str,
                                                       25: str,
                                                       42: str,
                                                       43: str,
                                                       47: str,
                                                       87: str,
                                                       89: str})

jersey_city.rename(columns={'Property Location': 'property_location', 
                            "Owner's Name": 'owners_name', 
                            "Owner's Mailing Address": 'owners_mailing_address', 
                            "City/State/Zip": "city_state_zip"}, inplace=True)

hackensack.rename(columns={'Property Location': 'property_location', 
                            "Owner's Name": 'owners_name', 
                            "Owner's Mailing Address": 'owners_mailing_address',
                            "City/State/Zip": "city_state_zip"}, inplace=True)


# title() case is necessary in order to return coordinates with Nominatim 
jersey_city['property_full_address'] = jersey_city.property_location.str.title() + ', Jersey City, NJ'
# we also need a column for the owner's full address
# city_state_zip refers to the owners_mailing_address, not property_location
jersey_city['owners_full_address'] = jersey_city.owners_mailing_address + ', ' + jersey_city.city_state_zip

In [3]:
def get_owner(address=str):
    owner = jersey_city[jersey_city.property_location == address.strip()]['owners_name'] 
    return owner

def get_properties(owner=str):
    unique_properties = jersey_city[jersey_city.owners_name == owner]['property_location'].unique()
    return unique_properties.tolist()

def get_number_properties_owned(owner=str):
    number_properties_owned = jersey_city[jersey_city.owners_name == owner]['property_location'].count()
    return number_properties_owned

In [4]:
jersey_city

,Municipality,Block,Lot,Qual,property_location,Property Class,owners_name,owners_mailing_address,city_state_zip,Sq. Ft.,...,Assessed.3,Latitude,Longitude,Neigh,VCS,StyDesc,Style,Unnamed: 90,property_full_address,owners_full_address
0,906,101.0,1.01,HM,1075 SECAUCUS RD.,1,"20 AQUARIUM DR.,LLC%NAT'L RETAIL",611 ROUTE 46 WEST,"HASBROUCK HEIGHTS, N.J. 07604",0,...,20733400.0,0,0,NaN,NaN,NaN,NaN,NaN,"1075 Secaucus Rd., Jersey City, NJ","611 ROUTE 46 WEST, HASBROUCK HEIGHTS, N.J. 07604"
1,906,101.0,2.00,HM,1045 SECAUCUS RD.,4A,PUBLIC SERVICE ELECTRIC & GAS CO.,80 PARK PL.TAX DEPT.T-6B,"NEWARK, N.J. 07102",0,...,227600.0,0,0,NaN,NaN,NaN,NaN,NaN,"1045 Secaucus Rd., Jersey City, NJ","80 PARK PL.TAX DEPT.T-6B, NEWARK, N.J. 07102"
2,906,101.0,3.00,HM,SECAUCUS ROAD,15C,UNITED STATES OF AMERICA,"475 L'ENFANT PLAZA, SW","WASHINGTON, D C 20260",0,...,4182300.0,0,0,NaN,NaN,NaN,NaN,NaN,"Secaucus Road, Jersey City, NJ","475 L'ENFANT PLAZA, SW, WASHINGTON, D C 20260"
3,906,101.0,5.00,HM,INSIDE SECAUCUS RD.,4A,HOLLAND CO.C/O PUB.SER.GAS&ELEC.CO.,80 PARK PL.-TAX DEPT.T-6B,"NEWARK, N.J. 07102",0,...,682400.0,0,0,NaN,NaN,NaN,NaN,NaN,"Inside Secaucus Rd., Jersey City, NJ","80 PARK PL.-TAX DEPT.T-6B, NEWARK, N.J. 07102"
4,906,101.0,6.00,HM,SECAUCUS RD,15C,UNITED STATES OF AMERICA,"475 L'ENFANT PLAZA, SW","WASHINGTON, D C 20260",0,...,7122400.0,0,0,NaN,NaN,NaN,NaN,NaN,"Secaucus Rd, Jersey City, NJ","475 L'ENFANT PLAZA, SW, WASHINGTON, D C 20260"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63829,906,99990.0,1.00,NaN,VARIOUS,6A,VERIZON-NJ C/O DUFF & PHELPS,P.O. BOX 2749,"ADDISON, TX 75001",0,...,64373216.0,0,0,NaN,NaN,NaN,NaN,NaN,"Various, Jersey City, NJ","P.O. BOX 2749, ADDISON, TX 75001"
63830,906,99990.0,2.00,NaN,VARIOUS,1,"TEXAS EASTERN TRANSMISSION,LP",P.O. BOX 2629,"ADDISON, TX 75001",0,...,22067400.0,0,0,NaN,NaN,NaN,NaN,NaN,"Various, Jersey City, NJ","P.O. BOX 2629, ADDISON, TX 75001"
63831,906,99990.0,3.00,NaN,VARIOUS,15F,"AIRBNB, INC",888 BRANNAN ST.,"SAN FRANCISCO, CA 94103",0,...,0.0,0,0,NaN,NaN,NaN,NaN,NaN,"Various, Jersey City, NJ","888 BRANNAN ST., SAN FRANCISCO, CA 94103"
63832,906,99990.0,4.00,NaN,VARIOUS,4B,TRANSCONTINENTAL GAS PIPE LINE,"P.O. BOX 2400, MD 46-4","TULSA, OK 74102",0,...,2286000.0,0,0,NaN,NaN,NaN,NaN,NaN,"Various, Jersey City, NJ","P.O. BOX 2400, MD 46-4, TULSA, OK 74102"


#### The following five cells are columns which can be deleted from the dataframe...
#### ...I needed to figure out their respective indices

In [5]:
jersey_city.columns[5]

'Property Class'

In [6]:
jersey_city.columns[9]

'Sq. Ft.'

In [7]:
jersey_city.columns[11:83]

Index(['Building Class', 'Prior Block', 'Prior Lot', 'Prior Qual', 'Updated',
       'Zone', 'Account', 'Mortgage Account', 'Bank Code', 'Sp Tax Cd',
       'Sp Tax Cd.1', 'Sp Tax Cd.2', 'Sp Tax Cd.3', 'Map Page',
       'Additional Lots', 'Land Desc', 'Building Desc', 'Class 4 Code',
       'Acreage', 'EPL Own', 'EPL Use', 'EPL Desc', 'EPL Statute', 'EPL Init',
       'EPL Further', 'EPL Facility Name', 'Taxes 1', 'Taxes 2', 'Taxes 3',
       'Taxes 4', 'Sale Date', 'Deed Book', 'Deed Page', 'Sale Price',
       'NU Code', 'Ratio', 'Type/Use', 'Year', 'Owner', 'Street',
       'City/State/Zip.1', 'Land Assmnt', 'Building Assmnt', 'Exempt',
       'Total Assmnt', 'Assessed', 'Year.1', 'Owner.1', 'Street.1',
       'City/State/Zip.2', 'Land Assmnt.1', 'Building Assmnt.1', 'Exempt.1',
       'Total Assmnt.1', 'Assessed.1', 'Year.2', 'Owner.2', 'Street.2',
       'City/State/Zip.3', 'Land Assmnt.2', 'Building Assmnt.2', 'Exempt.2',
       'Total Assmnt.2', 'Assessed.2', 'Year.3', 'Owner.3

In [8]:
jersey_city.columns[83]

'Assessed.3'

In [9]:
jersey_city.columns[86:91]

Index(['Neigh', 'VCS', 'StyDesc', 'Style', 'Unnamed: 90'], dtype='object')

## Does the owner live in the building? 
## Does the 'property location' match the 'owner's mailing address'?

In [10]:
# this method is inefficient, given we are searching 60k+ rows
jersey_city.property_location == jersey_city.owners_mailing_address

0        False
1        False
2        False
3        False
4        False
         ...  
63829    False
63830    False
63831    False
63832    False
63833    False
Length: 63834, dtype: bool

In [11]:
# numpy has better methods for quickly checking
# a quick look by wrapping the same equivalence in a numpy function to see if our intuition is correct
np.where(jersey_city.property_location == jersey_city.owners_mailing_address)

(array([   27,    29,    34, ..., 63776, 63796, 63797]),)

In [12]:
# owners associated with the most property addresses, in descending order
# note that these include non-residential, commerical, and industrial zoned properties 
jersey_city.owners_name.value_counts().head(50)

COA 99 HUDSON,LLC                      626
CITY OF JERSEY CITY                    325
75 PARK LANE, LLC                      215
95 VAN DAM URBAN RENEWAL, L.L.C.       129
CONSOLIDATED RAIL                      124
160 FIRST STREET URBAN RENEWAL, LLC    104
JERSEY CITY REDEVELOPMENT AGENCY        92
NEW JERSEY TRANSIT                      86
HOUSING AUTHORITY OF JERSEY CITY        74
LIBERTY HARBOR NORTH URBAN R., LLC      71
STATE OF N J DEPT OF ENV PROTECTION     57
BOARD OF EDUCATION OF J C               54
NJ DEPARTMENT OF TRANSPORTATION         51
300 COMMUNIPAW, INC.                    48
VILLAGE CONDOS III, LLC.                46
BERGEN AVE.ASSOC.C/O OSTROW             46
PUBLIC SERVICE ELECTRIC & GAS CO.       44
NEWPORT CENTRE                          40
N.J. DEPT. OF TRANSPORTATION            39
BLOCK 284 NORTH URBAN RENEWAL, LLC      33
AUDUBON AV.RLTY. %JASCO MANAGEMENT      32
CCA NEWPORT, INC.                       32
ERIE 10TH URBAN RENEWAL, L.L.C.         28
LIBERTY HAR

### property location matching the owners mailing address…can we safely assume they reside at the address?
### as many as 16544 owners reside in the building they own: 
### however they might not all be residential properties

In [13]:
jersey_city.loc[jersey_city.property_location == jersey_city.owners_mailing_address]

,Municipality,Block,Lot,Qual,property_location,Property Class,owners_name,owners_mailing_address,city_state_zip,Sq. Ft.,...,Assessed.3,Latitude,Longitude,Neigh,VCS,StyDesc,Style,Unnamed: 90,property_full_address,owners_full_address
27,906,201.0,7.0,NaN,817 TONNELE AVE.,1,"NELMIR REALTY, L.L.C.",817 TONNELE AVE.,"JERSEY CITY, NJ 07307",0,...,111900.0,0,0,NaN,NaN,NaN,NaN,NaN,"817 Tonnele Ave., Jersey City, NJ","817 TONNELE AVE., JERSEY CITY, NJ 07307"
29,906,201.0,9.0,NaN,823 TONNELE AVE.,4A,GERRY GAS SUPPLY INC.C/O LILH,823 TONNELE AVE.,"JERSEY CITY, N.J. 07307",0,...,1699500.0,0,0,NaN,NaN,NaN,NaN,NaN,"823 Tonnele Ave., Jersey City, NJ","823 TONNELE AVE., JERSEY CITY, N.J. 07307"
34,906,301.0,1.0,NaN,677 LIBERTY AVE.,2,"PEDDI, PRADEEP",677 LIBERTY AVE.,"JERSEY CITY, N.J. 07307",1600,...,416500.0,0,0,NaN,NaN,NaN,NaN,NaN,"677 Liberty Ave., Jersey City, NJ","677 LIBERTY AVE., JERSEY CITY, N.J. 07307"
35,906,301.0,2.0,NaN,675 LIBERTY AVE.,2,"PAREJA, HENRY A. & MIRYAM C.",675 LIBERTY AVE.,"JERSEY CITY, N.J. 07307",1616,...,363300.0,0,0,NaN,NaN,NaN,NaN,NaN,"675 Liberty Ave., Jersey City, NJ","675 LIBERTY AVE., JERSEY CITY, N.J. 07307"
36,906,301.0,3.0,NaN,673 LIBERTY AVE.,2,"HIRPARA, PRAVIN",673 LIBERTY AVE.,"JERSEY CITY, NJ 07307",1600,...,364900.0,0,0,NaN,NaN,NaN,NaN,NaN,"673 Liberty Ave., Jersey City, NJ","673 LIBERTY AVE., JERSEY CITY, NJ 07307"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63740,906,30304.0,14.0,NaN,100 GARFIELD AVE.,2,"SERVIDA, AUREA",100 GARFIELD AVE.,"JERSEY CITY, NJ 07305",1882,...,271000.0,0,0,NaN,NaN,NaN,NaN,NaN,"100 Garfield Ave., Jersey City, NJ","100 GARFIELD AVE., JERSEY CITY, NJ 07305"
63742,906,30304.0,16.0,NaN,104 GARFIELD AVE.,2,"RODRIGUEZ, BENJAMIN",104 GARFIELD AVE.,"JERSEY CITY, N.J. 07305",2412,...,268200.0,0,0,NaN,NaN,NaN,NaN,NaN,"104 Garfield Ave., Jersey City, NJ","104 GARFIELD AVE., JERSEY CITY, N.J. 07305"
63776,906,30306.0,8.0,NaN,100 SUMMIT PLACE,4B,"SUMMIT/GREENWICH URBAN RENEWAL, LLC",100 SUMMIT PLACE,"JERSEY CITY, NJ 07305",0,...,13139700.0,0,0,NaN,NaN,NaN,NaN,NaN,"100 Summit Place, Jersey City, NJ","100 SUMMIT PLACE, JERSEY CITY, NJ 07305"
63796,906,30307.0,10.0,NaN,109 PORT JERSEY BLVD.,4B,DOMISA LLC,109 PORT JERSEY BLVD.,"JERSEY CITY, N.J. 07305",0,...,2863800.0,0,0,NaN,NaN,NaN,NaN,NaN,"109 Port Jersey Blvd., Jersey City, NJ","109 PORT JERSEY BLVD., JERSEY CITY, N.J. 07305"


In [14]:
# number of units at this propety 
jersey_city[jersey_city.owners_name == '75 PARK LANE, LLC']['property_location'].count()

215

In [15]:
# we know these are units when we add the .unique() accessor 
jersey_city[jersey_city.owners_name == '75 PARK LANE, LLC']['property_location'].unique()

array(['75 PARK LANE SOUTH'], dtype=object)

In [16]:
# As we'll see below, many of the addresses and owners are spelled incorrectly 
len(jersey_city['owners_name'].unique())

49247

In [17]:
len(jersey_city['property_location'].unique())

37842

## How many LLCs have "UR" or "Urban Renewal" in the owner's name?

In [18]:
# More examples of how unclean the data is, even on a state database
jersey_city[jersey_city.owners_name.str.contains('URBAN RENEWAL')]['owners_name'].unique()

array(['HEIGHTS URBAN RENEWAL SENIOR H.',
       'RATAN JERSEY CITY URBAN RENEWAL,LLC',
       'BRASS WORKS URBAN RENEWAL CO., LLC',
       'HUDSON PALISADES URBAN RENEWAL,LLC',
       '364 NINTH STREET URBAN RENEWAL, LLC',
       'NINTH STREET TWO URBAN RENEWAL,LLC',
       'TOWER EAST URBAN RENEWAL COMPANY',
       'JAMES MONROE URBAN RENEWAL CO.', '25 RIVER DR.SO.URBAN RENEWAL',
       'SENATE PLACE URBAN RENEWAL, LLC',
       'H.P. LINCOLN URBAN RENEWAL COMPANY',
       'ERIE 10TH URBAN RENEWAL, L.L.C.', '9TH STREET URBAN RENEWAL,LLC',
       'VAN WAGENEN II URBAN RENEWAL, LLC.',
       '17-19 DIVISION ST URBAN RENEWAL,LLC',
       '380 NEWARK REALTY URBAN RENEWAL LLC',
       'BLOCK 284 NORTH URBAN RENEWAL, LLC',
       '500 MANILA AVE.URBAN RENEWAL, LLC',
       '500 MANILA AVE.URBAN RENEWAL LLC',
       'KRE HAMILTON URBAN RENEWAL LLC',
       'VAN WAGENEN I URBAN RENEWAL, LLC.',
       '160 FIRST STREET URBAN RENEWAL, LLC',
       'PS FIRST HUDSON URBAN RENEWAL LLC',
       '14

In [19]:
jersey_city[jersey_city.owners_name.str.contains(' UR ')]['owners_name'].unique()

array(['SUMMIT PLAZA ASSOCIATES, UR LTD PTN',
       'PADUA COURT UR C/O INSPIRED VISION',
       'PADUA COURT UR C/O INSPRED VISION',
       'CAL-HARBOR SO. PIER UR ASC %M CALI'], dtype=object)

In [20]:
jersey_city[jersey_city.owners_name == 'COA 99 HUDSON,LLC']['property_location'].unique()

array(['99 HUDSON ST.'], dtype=object)

In [21]:
%%time
# a smaller dataset to view inconsistencies
# are there duplicate addresses in this subset of data?
# are the addresses accurate? 

# a copy() is used to avoid writing on the original dataframe
js_ha = jersey_city[jersey_city.owners_name == 'HOUSING AUTHORITY OF JERSEY CITY'].copy()
js_ha.reset_index(inplace=True, drop=True)
js_ha['gcode'] = js_ha.property_full_address.apply(geolocator.geocode)

city_of_js = jersey_city[jersey_city.owners_name == 'CITY OF JERSEY CITY'].copy()
city_of_js.reset_index(inplace=True, drop=True)
city_of_js['gcode'] = city_of_js.property_full_address.apply(geolocator.geocode)

urban_renewal = jersey_city[jersey_city.owners_name.str.contains('URBAN RENEWAL')].copy()
urban_renewal.reset_index(inplace=True, drop=True)
urban_renewal['gcode'] = urban_renewal.property_full_address.apply(geolocator.geocode)

CPU times: user 1.53 s, sys: 234 ms, total: 1.77 s
Wall time: 7min 8s


In [22]:
# for now, we'll drop the None values in the gcode column to make our lives easier 
# this does not however solve the issue of duplicate values in that column
js_ha.dropna(axis=0, subset=['gcode'], inplace=True)
city_of_js.dropna(axis=0, subset=['gcode'], inplace=True)
urban_renewal.dropna(axis=0, subset=['gcode'], inplace=True)

# since the dataframe already contains Latitude and Longitude columns, we can write to them directly
js_ha['Latitude'] = [g.latitude for g in js_ha.gcode]
js_ha['Longitude'] = [g.longitude for g in js_ha.gcode]

city_of_js['Latitude'] = [g.latitude for g in city_of_js.gcode]
city_of_js['Longitude'] = [g.longitude for g in city_of_js.gcode]

urban_renewal['Latitude'] = [g.latitude for g in urban_renewal.gcode]
urban_renewal['Longitude'] = [g.longitude for g in urban_renewal.gcode]

In [23]:
# a less tedious way to execute this might be drop ranges of columns instead of doing it all manually
js_ha.drop(columns=['Municipality', 
                    'Block', 
                    'Lot', 
                    'Qual', 
                    'Property Class', 
                    'Sq. Ft.', 
                    'Building Class', 
                    'Prior Block', 
                    'Prior Lot', 
                    'Prior Qual', 
                    'Updated', 
                    'Zone', 
                    'Account', 
                    'Mortgage Account', 
                    'Bank Code', 
                    'Sp Tax Cd', 
                    'Sp Tax Cd.1', 
                    'Sp Tax Cd.2', 
                    'Sp Tax Cd.3', 
                    'Map Page',
                    'Additional Lots', 
                    'Land Desc', 
                    'Building Desc', 
                    'Class 4 Code', 
                    'Acreage', 
                    'EPL Own', 
                    'EPL Use', 
                    'EPL Desc', 
                    'EPL Statute', 
                    'EPL Init', 
                    'EPL Further', 
                    'EPL Facility Name', 
                    'Taxes 1', 
                    'Taxes 2', 
                    'Taxes 3', 
                    'Taxes 4', 
                    'Sale Date', 
                    'Deed Book', 
                    'Deed Page', 
                    'Sale Price', 
                    'NU Code', 
                    'Ratio', 
                    'Type/Use', 
                    'Year', 
                    'Owner', 
                    'Street', 
                    'City/State/Zip.1',
                    'Land Assmnt', 
                    'Building Assmnt', 
                    'Exempt', 
                    'Total Assmnt', 
                    'Assessed',
                    'Year.1', 
                    'Owner.1', 
                    'Street.1', 
                    'City/State/Zip.2', 
                    'Land Assmnt.1',
                    'Building Assmnt.1', 
                    'Exempt.1', 
                    'Total Assmnt.1', 
                    'Assessed.1',
                    'Year.2', 
                    'Owner.2', 
                    'Street.2', 
                    'City/State/Zip.3', 
                    'Land Assmnt.2',
                    'Building Assmnt.2', 
                    'Exempt.2', 
                    'Total Assmnt.2', 
                    'Assessed.2',
                    'Year.3', 
                    'Owner.3', 
                    'Street.3', 
                    'City/State/Zip.4', 
                    'Land Assmnt.3',
                    'Building Assmnt.3', 
                    'Exempt.3', 
                    'Total Assmnt.3', 
                    'Assessed.3', 
                    'Neigh', 
                    'VCS', 
                    'StyDesc', 
                    'Style',
                    'Unnamed: 90'], inplace=True) 

In [24]:
# a less tedious way to execute this might be drop ranges of columns instead of doing it all manually
city_of_js.drop(columns=['Municipality', 
                    'Block', 
                    'Lot', 
                    'Qual', 
                    'Property Class', 
                    'Sq. Ft.', 
                    'Building Class', 
                    'Prior Block', 
                    'Prior Lot', 
                    'Prior Qual', 
                    'Updated', 
                    'Zone', 
                    'Account', 
                    'Mortgage Account', 
                    'Bank Code', 
                    'Sp Tax Cd', 
                    'Sp Tax Cd.1', 
                    'Sp Tax Cd.2', 
                    'Sp Tax Cd.3', 
                    'Map Page',
                    'Additional Lots', 
                    'Land Desc', 
                    'Building Desc', 
                    'Class 4 Code', 
                    'Acreage', 
                    'EPL Own', 
                    'EPL Use', 
                    'EPL Desc', 
                    'EPL Statute', 
                    'EPL Init', 
                    'EPL Further', 
                    'EPL Facility Name', 
                    'Taxes 1', 
                    'Taxes 2', 
                    'Taxes 3', 
                    'Taxes 4', 
                    'Sale Date', 
                    'Deed Book', 
                    'Deed Page', 
                    'Sale Price', 
                    'NU Code', 
                    'Ratio', 
                    'Type/Use', 
                    'Year', 
                    'Owner', 
                    'Street', 
                    'City/State/Zip.1',
                    'Land Assmnt', 
                    'Building Assmnt', 
                    'Exempt', 
                    'Total Assmnt', 
                    'Assessed',
                    'Year.1', 
                    'Owner.1', 
                    'Street.1', 
                    'City/State/Zip.2', 
                    'Land Assmnt.1',
                    'Building Assmnt.1', 
                    'Exempt.1', 
                    'Total Assmnt.1', 
                    'Assessed.1',
                    'Year.2', 
                    'Owner.2', 
                    'Street.2', 
                    'City/State/Zip.3', 
                    'Land Assmnt.2',
                    'Building Assmnt.2', 
                    'Exempt.2', 
                    'Total Assmnt.2', 
                    'Assessed.2',
                    'Year.3', 
                    'Owner.3', 
                    'Street.3', 
                    'City/State/Zip.4', 
                    'Land Assmnt.3',
                    'Building Assmnt.3', 
                    'Exempt.3', 
                    'Total Assmnt.3', 
                    'Assessed.3', 
                    'Neigh', 
                    'VCS', 
                    'StyDesc', 
                    'Style',
                    'Unnamed: 90'], inplace=True) 

In [25]:
urban_renewal.drop(columns=['Municipality', 
                    'Block', 
                    'Lot', 
                    'Qual', 
                    'Property Class', 
                    'Sq. Ft.', 
                    'Building Class', 
                    'Prior Block', 
                    'Prior Lot', 
                    'Prior Qual', 
                    'Updated', 
                    'Zone', 
                    'Account', 
                    'Mortgage Account', 
                    'Bank Code', 
                    'Sp Tax Cd', 
                    'Sp Tax Cd.1', 
                    'Sp Tax Cd.2', 
                    'Sp Tax Cd.3', 
                    'Map Page',
                    'Additional Lots', 
                    'Land Desc', 
                    'Building Desc', 
                    'Class 4 Code', 
                    'Acreage', 
                    'EPL Own', 
                    'EPL Use', 
                    'EPL Desc', 
                    'EPL Statute', 
                    'EPL Init', 
                    'EPL Further', 
                    'EPL Facility Name', 
                    'Taxes 1', 
                    'Taxes 2', 
                    'Taxes 3', 
                    'Taxes 4', 
                    'Sale Date', 
                    'Deed Book', 
                    'Deed Page', 
                    'Sale Price', 
                    'NU Code', 
                    'Ratio', 
                    'Type/Use', 
                    'Year', 
                    'Owner', 
                    'Street', 
                    'City/State/Zip.1',
                    'Land Assmnt', 
                    'Building Assmnt', 
                    'Exempt', 
                    'Total Assmnt', 
                    'Assessed',
                    'Year.1', 
                    'Owner.1', 
                    'Street.1', 
                    'City/State/Zip.2', 
                    'Land Assmnt.1',
                    'Building Assmnt.1', 
                    'Exempt.1', 
                    'Total Assmnt.1', 
                    'Assessed.1',
                    'Year.2', 
                    'Owner.2', 
                    'Street.2', 
                    'City/State/Zip.3', 
                    'Land Assmnt.2',
                    'Building Assmnt.2', 
                    'Exempt.2', 
                    'Total Assmnt.2', 
                    'Assessed.2',
                    'Year.3', 
                    'Owner.3', 
                    'Street.3', 
                    'City/State/Zip.4', 
                    'Land Assmnt.3',
                    'Building Assmnt.3', 
                    'Exempt.3', 
                    'Total Assmnt.3', 
                    'Assessed.3', 
                    'Neigh', 
                    'VCS', 
                    'StyDesc', 
                    'Style',
                    'Unnamed: 90'], inplace=True) 

In [26]:
test_data = js_ha.append([city_of_js, urban_renewal])
test_data.reset_index(inplace=True, drop=True)

In [39]:
def test_get_owner(address=str):
    owner = test_data[test_data.property_location == address.strip()]['owners_name']
    return owner

def test_get_properties(owner=str):
    unique_properties = test_data[test_data.owners_name == owner]['property_location'].unique()
    return unique_properties.tolist()

def test_get_number_owned(owner=str):
    number_owned = test_data[test_data.owners_name == owner]['property_location'].count()
    return number_owned

def test_get_count(address=str):
    count = test_data[test_data.property_location == address]['property_location'].count()
    return count


In [43]:
test_data['asc_properties'] = [test_get_properties(owner) for owner in test_data.owners_name]
test_data['num_asc_properties'] = [test_get_number_owned(owner) for owner in test_data.owners_name]
# ATTENTION! This is not the correct way to derive the number of units at a given address
# I am use this to generate data to build functionality for the landlord-app
test_data['units'] = [test_get_count(address) for address in test_data.property_location]

In [76]:
get_properties('NJ PENELOPE LLC.')

['272 OGDEN AVE.',
 '31 SHERMAN AVE.',
 '123 WEBSTER AVE.',
 '39.5 WAVERLY STREET',
 '19 WAVERLY ST.',
 '656 JERSEY AVE.',
 '660 JERSEY AVE.',
 '279 EIGHTH ST.',
 '151 COLES ST.',
 '511 MANILA AVE.',
 '286 FIFTH ST.',
 '220.5 THIRD STREET',
 '2 HAMPTON CT. TER.',
 '1 HAMPTON CT. TER.',
 '101 MERCER ST.',
 '459 JERSEY AVE.',
 '328 YORK ST.',
 '295 VARICK ST.',
 '137 MERCER ST.',
 '244 MONTGOMERY ST.',
 '268 BARROW ST.',
 '317 HALLADAY ST.',
 '14 MONTICELLO AVE.',
 '69 WILLIAMS AVE.']

In [77]:
get_number_properties_owned('NJ PENELOPE LLC.')

24

In [59]:
get_owner('272 OGDEN AVE.')

7512    NJ PENELOPE LLC.
Name: owners_name, dtype: object

In [32]:
get_owner('143 CHAPEL AVE	')

604    HUDSON MAIN URBAN RENEWAL,LLC
605    HUDSON MAIN URBAN RENEWAL,LLC
Name: owners_name, dtype: object

In [45]:
test_data.to_csv('test_data.csv', index=None)

In [22]:
#  finding the indices of the properties which were not geocoded
[i for i,e in enumerate([geolocator.geocode(address) for address in js_ha.property_full_address]) if e == None]

[0, 4, 14, 15, 23, 24, 25, 26, 29, 30, 35, 36, 37, 39, 45, 54]

### We know the addresses are unclean when we substitute parts of the address
### For example, "Sixteenth" for "16th": 

In [162]:
geolocator.geocode('235 16th St. Jersey City, NJ 07307')

Location(235, 16th Street, Jersey City, Hudson County, New Jersey, 07310, United States, (40.73314597560976, -74.04304924390243, 0.0))

In [168]:
js_ha.property_full_address[23]

'9-31 Wilmot Ave. Jersey City, NJ 07307'

In [123]:
geolocator.geocode('61 Merritt St. Jersey City, NJ 07307').point

Point(40.6888175, -74.0973369, 0.0)

In [118]:
jersey_city[jersey_city.owners_name == '160 FIRST STREET URBAN RENEWAL, LLC']['property_location']

23540    160 FIRST ST.
23541    160 FIRST ST.
23544    160 FIRST ST.
23545    160 FIRST ST.
23556    160 FIRST ST.
             ...      
23857    160 FIRST ST.
23858    160 FIRST ST.
23859    160 FIRST ST.
23860    160 FIRST ST.
23861    160 FIRST ST.
Name: property_location, Length: 104, dtype: object

In [14]:
hackensack.owners_name.value_counts().head(50)

CITY OF HACKENSACK                     69
WORLD PLAZA AC, LLC                    29
ESSEX COURT REALTY CORP.               28
HACKENSACK DEVELOPERS, LLC             25
TERRACE SQUARE CONDOS % J LOMBARDO     21
N Y S & W R R C/O N. STECKLER          19
HACK & N.Y. R.R. CO C/O LAND & TAX     19
SKYVIEW AT HACKENSACK, LLC             19
COUNTY OF BERGEN                       16
WORLD PLAZA PROPERTIES, LLC            13
ATTESSA PROPERTIES LLC                 13
HEKEMIAN,SAMUEL %THE S HEKEMIAN GRP    12
BD OF ED CITY OF HACK                  12
ELYASH, ADELAIDA                       10
FAIRLEIGH DICKINSON UNIVERSITY          9
400 E MAIN STREET, LLC                  9
NEW HOPE BAPTIST CHURCH                 8
RUDDOCK, WAYNE & ROSA                   7
DI CAROLIS REALTY CO.                   7
JACKSON PINK LLC                        7
NESS REALTY 1 LLC                       7
MAIN PORTFOLIO LLC                      6
US BANK TRUST NA TRSTE                  6
CYPRUS HOLDING COMPANY LLC        